### 자동 미분 (torch.autograd)

- PyTorch의 autograd 는 신경망 훈련을 지원하는 자동 미분 가능
- torch.autograd 동작 방법
    - 텐서에 .requires_grad 속성을 True로 설정하면, 이후의 텐서 모든 연산들을 추적함
    - 텐서.backward() 를 호출하면, 연산에 연결된 각 텐서들의 미분 값을 계산하여, 각 텐서 객체에 .grad에 저장
        - .requires_grad_()는 연결된 Tensor로부터의 계산된 자동미분 값을, 다시 현 텐서부터 시작하도록 만듦

### 신경망 동작 이해

- 모델 및 데이터 생성
- forward pass로 입력 데이터를 모델에 넣어서 예측값 계산
- 예측값과 실제값의 차이를 loss function 으로 계산
- backward pass 로 각 모델 파라미터를 loss 값 기반 미분하여 저장
- optimizer 로 모델 파라미터의 최적값을 찾기 위해, 파라미터 값 업데이트

- 텐서에 .requires_grad 속성을 True 로 설정
- .requires_grad 속성이 True 로 설정되면, 텐서의 모든 연산 추적을 위해, 내부적으로 방향성 비순환 그래프(DAG : Directed Acyclic Graph)를 동적 구성
    - 방향성 비순환 그래프(DAG)의 leaf 노드는 입력 텐서이고, root 노드는 결과 텐서가 됨

In [30]:
import torch

x = torch.rand(1, requires_grad=True)
y = torch.rand(1)
y.requires_grad = True
loss = y - x

### 🔁 텐서의 `.backward()` 동작 설명

- `tensor.backward()` 를 호출하면,  
  연산에 연결된 각 텐서들의 **미분 값(gradient)**을 자동으로 계산하여  
  각 텐서 객체의 `.grad` 속성에 저장된다.

---

### 🎯 예시

$$
\frac{\partial \text{Loss}}{\partial x} = -1, \quad \frac{\partial \text{Loss}}{\partial y} = 1
$$

> 즉, Loss를 기준으로 각 입력값에 대한 **기울기(gradient)**가 계산되어  
> `.grad` 속성에 자동으로 저장된다.

In [31]:
loss.backward()
print(x.grad, y.grad)

tensor([-1.]) tensor([1.])


In [32]:
x = torch.ones(4)
y = torch.zeros(3)
W = torch.rand(4, 3, requires_grad=True)
b = torch.rand(3, requires_grad=True)
z = torch.matmul(x,W) + b
print(W, b, z)

tensor([[0.5649, 0.3083, 0.0680],
        [0.4738, 0.4295, 0.5688],
        [0.9297, 0.4158, 0.8811],
        [0.7368, 0.5742, 0.1030]], requires_grad=True) tensor([0.5654, 0.2019, 0.7719], requires_grad=True) tensor([3.2705, 1.9297, 2.3927], grad_fn=<AddBackward0>)


In [33]:
import torch.nn.functional as F

loss = F.mse_loss(z, y)
loss.backward()
print(loss, W.grad, b.grad)


tensor(6.7151, grad_fn=<MseLossBackward0>) tensor([[2.1804, 1.2864, 1.5952],
        [2.1804, 1.2864, 1.5952],
        [2.1804, 1.2864, 1.5952],
        [2.1804, 1.2864, 1.5952]]) tensor([2.1804, 1.2864, 1.5952])


In [34]:
threshold = 0.1
learning_rate = 0.1
iteration_num = 0

while loss > threshold :
    iteration_num += 1
    W = W - learning_rate * W.grad
    b = b - learning_rate * b.grad
    print(iteration_num, loss, z, y)
    
    # detach_() : 텐서를 기존 방향성 비순환 그래프(DAG : Directed Acyclid Graph) 로부터 끊음
    # .requires_grad(True) : 연결된 Tensor 로부터의 계산된 자동미분 값을, 다시 현 텐서부터 시작하도록 만듦
    W.detach_().requires_grad_(True)
    b.detach_().requires_grad_(True)

    z = torch.matmul(x, W) + b
    loss = F.mse_loss(z, y)
    loss.backward()

print(iteration_num + 1, loss, z, y)

1 tensor(6.7151, grad_fn=<MseLossBackward0>) tensor([3.2705, 1.9297, 2.3927], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
2 tensor(2.9845, grad_fn=<MseLossBackward0>) tensor([2.1804, 1.2864, 1.5952], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
3 tensor(1.3264, grad_fn=<MseLossBackward0>) tensor([1.4536, 0.8576, 1.0634], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
4 tensor(0.5895, grad_fn=<MseLossBackward0>) tensor([0.9691, 0.5718, 0.7090], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
5 tensor(0.2620, grad_fn=<MseLossBackward0>) tensor([0.6460, 0.3812, 0.4726], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
6 tensor(0.1165, grad_fn=<MseLossBackward0>) tensor([0.4307, 0.2541, 0.3151], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
7 tensor(0.0518, grad_fn=<MseLossBackward0>) tensor([0.2871, 0.1694, 0.2101], grad_fn=<AddBackward0>) tensor([0., 0., 0.])


### Optimizer 와 경사 하강법

- 최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해서 모델 매개변수를 조정하는 과정으로 Optimizer는 최적화 알고리즘을 의미합니다.
- 대표적인 최적화 알고리즘에는 확률적 경사하강법(SGD: Stochastic Gradient Descent)
- PyTorch 에는 모델과 데이터 타입에 따라, 보다 좋은 성능을 제공하는 ADAM 이나 RMSProp과 같은 다양한 옵티마이저가 존재합니다.

In [35]:
import torch

w = torch.tensor(4.0, requires_grad=True)
z = 2 * w
z.backward()
print(w.grad)

z = 2 * w
z.backward()
print(w.grad)

z = 2 * w
z.backward()
print(w.grad)

tensor(2.)
tensor(4.)
tensor(6.)


### SGD 경사하강법 Optimizer 적용

In [36]:
import torch

x = torch.ones(4)
y = torch.zeros(3)

W = torch.rand(4, 3, requires_grad=True)
b = torch.rand(3, requires_grad=True)

In [37]:
learning_rate = 0.01
optimizer = torch.optim.SGD([W, b], lr=learning_rate)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [ ]:
nb_epochs = 300  # 원하는 만큼 경사하강법 반복
for epoch in range(nb_epochs + 1):
    
    z = torch.matmul(x, W) + b
    loss = F.mse_loss(z, y)
    
    optimizer.zero_grad() # 기울기 초기화
    loss.backward() # 기울기 계산
    optimizer.step() # 파라미터 업데이트 
    
    if epoch % 100 == 0:
        print(epoch, nb_epochs, W, b, loss)

0 300 tensor([[0.3289, 0.2782, 0.7777],
        [0.6289, 0.9512, 0.1976],
        [0.8176, 0.2167, 0.3775],
        [0.8384, 0.0892, 0.3002]], requires_grad=True) tensor([0.9355, 0.1868, 0.7305], requires_grad=True) tensor(7.5781, grad_fn=<MseLossBackward0>)
100 300 tensor([[-0.3571, -0.0546,  0.3171],
        [-0.0570,  0.6184, -0.2630],
        [ 0.1316, -0.1161, -0.0832],
        [ 0.1525, -0.2437, -0.1604]], requires_grad=True) tensor([ 0.2496, -0.1460,  0.2698], requires_grad=True) tensor(0.0086, grad_fn=<MseLossBackward0>)
200 300 tensor([[-0.3802, -0.0658,  0.3016],
        [-0.0801,  0.6072, -0.2785],
        [ 0.1085, -0.1273, -0.0987],
        [ 0.1294, -0.2549, -0.1759]], requires_grad=True) tensor([ 0.2265, -0.1573,  0.2543], requires_grad=True) tensor(9.7781e-06, grad_fn=<MseLossBackward0>)
300 300 tensor([[-0.3810, -0.0662,  0.3010],
        [-0.0809,  0.6068, -0.2791],
        [ 0.1078, -0.1277, -0.0992],
        [ 0.1286, -0.2552, -0.1765]], requires_grad=True) tensor([

### SGD 경사하강법 Optimizer 적용 (PyTorch 신경망 모델 클래스 기반)

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegressionModel(4,3)
model

LinearRegressionModel(
  (linear): Linear(in_features=4, out_features=3, bias=True)
)

In [40]:
x = torch.ones(4)
y = torch.zeros(3)

learning_rate = 0.01
nb_epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(nb_epochs + 1):
    
    pred = model(x)
    loss = F.mse_loss(pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [41]:
print(loss)
for param in model.parameters():
    print(param)

tensor(1.5065e-13, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[ 0.2709, -0.2626,  0.2644, -0.3309],
        [ 0.5202,  0.0626, -0.3250, -0.3387],
        [-0.2581, -0.2793,  0.4233,  0.1462]], requires_grad=True)
Parameter containing:
tensor([ 0.0583,  0.0808, -0.0321], requires_grad=True)
